In [2]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
df1 = pd.read_csv('../csv/keisuke1.csv',header=0)
df2 = pd.read_csv('../csv/keisuke2.csv',header=0)
df3 = pd.read_csv('../csv/keisuke3.csv',header=0)
df4 = pd.read_csv('../csv/keisuke4.csv',header=0)

In [4]:
df1 = df1.drop(df1[df1.STATE==1].index)
df2 = df2.drop(df2[df2.STATE==1].index)
df3 = df3.drop(df3[df3.STATE==1].index)
df4 = df4.drop(df4[df4.STATE==1].index)

In [5]:
df1.head()

,SEC,GSR,STATE
60,60,384,2
61,61,389,2
62,62,395,2
63,63,384,2
64,64,366,2


In [6]:
def create_data(X,Y):
    time_step=10;
    Xs = []
    Ys = np.zeros((X.size-time_step,2))
    for i in range(0,X.size-time_step,1):
        tmp = X.iloc[i:(i+time_step)]
        labels = Y.iloc[i:(i+time_step)]
        Xs.append(tmp)
        if stats.mode(labels)[0][0] == 2:
            Ys[i][0] = 1
        else:
            Ys[i][1] = 1
        
    return np.array(Xs),np.array(Ys)

In [7]:
#TRAIN
X_train,Y_train= create_data(df1['GSR'],df1['STATE'])
tmp1, tmp2 = create_data(df2['GSR'],df2['STATE'])
X_train = np.concatenate((X_train,tmp1),axis=0)
Y_train = np.concatenate((Y_train,tmp2),axis=0)
tmp1, tmp2 = create_data(df3['GSR'],df3['STATE'])
X_train = np.concatenate((X_train,tmp1),axis=0)
Y_train = np.concatenate((Y_train,tmp2),axis=0)

X_train = np.reshape(X_train,[1,X_train.shape[0],X_train.shape[1]])
Y_train = np.reshape(Y_train,[1,Y_train.shape[0],Y_train.shape[1]])

In [8]:
#TEST
X_test, Y_test = create_data(df4['GSR'],df4['STATE'])
X_test = np.reshape(X_test,[1,X_test.shape[0],X_test.shape[1]])
Y_test = np.reshape(Y_test,[1,Y_test.shape[0],Y_test.shape[1]])

In [9]:
print(X_train.shape,Y_train.shape)

(1, 1050, 10) (1, 1050, 2)


In [10]:
X_train

array([[[384, 389, 395, ..., 346, 347, 349],
        [389, 395, 384, ..., 347, 349, 356],
        [395, 384, 366, ..., 349, 356, 361],
        ...,
        [325, 320, 317, ..., 327, 324, 319],
        [320, 317, 317, ..., 324, 319, 316],
        [317, 317, 317, ..., 319, 316, 313]]])

In [11]:
Y_train

array([[[1., 0.],
        [1., 0.],
        [1., 0.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]]])

In [12]:
model = keras.Sequential()

In [13]:
model.add(keras.layers.Embedding(input_dim=1, output_dim=2))
model.add(keras.layers.LSTM(16, return_sequences=False, input_shape=(10,)))
model.add(keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
             metrics=['acc'])

In [14]:
history = model.fit(X_train,
                   Y_train,
                   epochs=30,
                   shuffle=False)

Epoch 1/30


ValueError: in user code:

    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.8/dist-packages/tensorflow/python/keras/engine/training.py:788 run_step  **
        outputs = model.t